In [6]:
import cv2
import os

# Specify the correct paths to the Haar cascade XML files
cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
eye_cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_eye.xml"

# Load the classifiers
detector = cv2.CascadeClassifier(cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

if detector.empty() or eye_cascade.empty():
    print("Error: Could not load Haar cascades. Check the file paths.")
else:
    print("Haar cascades loaded successfully!")

Haar cascades loaded successfully!


In [10]:
import cv2
import time
import numpy as np
import os

# Specify the correct paths to the Haar cascade XML files
cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
eye_cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_eye.xml"

# Load the classifiers
detector = cv2.CascadeClassifier(cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

if detector.empty() or eye_cascade.empty():
    print("Error: Could not load Haar cascades. Check the file paths.")
else:
    print("Haar cascades loaded successfully!")

# Load the pre-trained classifiers for face and eyes
# detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
# eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

def detect_pupil(eye_region):
    """Detects the pupil in the eye region using thresholding and contour detection."""
    gray = cv2.cvtColor(eye_region, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV)
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        return int(x), int(y), int(radius)
    return None

# Open webcam
cap = cv2.VideoCapture(0)
prev_time = time.time()
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))
    
    for (x, y, w, h) in faces:
        face_region = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(face_region, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))
        
        for (ex, ey, ew, eh) in eyes:
            eye_roi = face_region[ey:ey+eh, ex:ex+ew]
            pupil_center = detect_pupil(eye_roi)
            
            if pupil_center:
                px, py, pr = pupil_center
                cv2.circle(eye_roi, (px, py), pr, (0, 255, 0), 2)
            
            cv2.rectangle(face_region, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)
    
    # Calculate frame rate
    frame_count += 1
    current_time = time.time()
    fps = frame_count / (current_time - prev_time)
    
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow("Eye Tracking", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Haar cascades loaded successfully!


In [ ]:
cv2.destroyAllWindows()

In [2]:
print(cv2.__file__)

/opt/anaconda3/envs/cv-env/lib/python3.13/site-packages/cv2/__init__.py


In [ ]:
import cv2
import numpy as np
import time

# Haar cascade paths
cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
eye_cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_eye.xml"

# Load classifiers
face_cascade = cv2.CascadeClassifier(cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

# Open webcam
cap = cv2.VideoCapture(0)
prev_time = time.time()
frame_count = 0

def detect_pupil(eye_roi):
    """Detect pupil using HoughCircles"""
    gray = cv2.cvtColor(eye_roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)  # Reduce noise
    
    # Adaptive thresholding to handle lighting variations
    _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV)
    
    # Detect circles using HoughCircles
    circles = cv2.HoughCircles(thresh, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=50, param2=20, minRadius=5, maxRadius=30)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            return (i[0], i[1], i[2])  # x, y, radius
    return None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.2, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))

        for (ex, ey, ew, eh) in eyes:
            eye_roi = face_roi[ey:ey+eh, ex:ex+ew]
            pupil_center = detect_pupil(eye_roi)

            if pupil_center:
                px, py, pr = pupil_center
                cv2.circle(eye_roi, (px, py), pr, (0, 255, 0), 2)  # Draw circle on pupil

            cv2.rectangle(face_roi, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)  # Draw rectangle around eyes

    # Calculate frame rate
    frame_count += 1
    current_time = time.time()
    fps = frame_count / (current_time - prev_time)

    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Eye Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2025-03-13 10:06:48.082 python[19626:444760] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-13 10:06:48.082 python[19626:444760] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 

In [6]:
import cv2
import numpy as np
import time

# Load Haar Cascade models for face and eyes
face_cascade = cv2.CascadeClassifier("/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_eye.xml")

# Open webcam
cap = cv2.VideoCapture(0)
prev_time = time.time()
frame_count = 0

def preprocess_eye(eye_roi):
    """Preprocess eye image: Convert to grayscale, apply blurring & thresholding"""
    gray = cv2.cvtColor(eye_roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)  # Reduce noise
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)  # Adaptive thresholding
    return thresh

def detect_pupil(eye_roi):
    """Detect the pupil using contours and HoughCircles"""
    processed_eye = preprocess_eye(eye_roi)

    # Find edges
    edges = cv2.Canny(processed_eye, 50, 150)

    # Find contours and filter by area (removes false positives)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)  # Sort by largest contour

    for contour in contours:
        if 10 < cv2.contourArea(contour) < 150:  # Filter small & large blobs
            (x, y), radius = cv2.minEnclosingCircle(contour)
            return int(x), int(y), int(radius)

    return None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.2, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))

        for (ex, ey, ew, eh) in eyes:
            eye_roi = face_roi[ey:ey+eh, ex:ex+ew]
            pupil_center = detect_pupil(eye_roi)

            if pupil_center:
                px, py, pr = pupil_center
                cv2.circle(eye_roi, (px, py), pr, (0, 255, 0), 2)  # Draw circle on pupil

            cv2.rectangle(face_roi, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)  # Draw rectangle around eyes

    # Calculate frame rate
    frame_count += 1
    current_time = time.time()
    fps = frame_count / (current_time - prev_time)

    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Optimized Eye Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [5]:
import cv2
import numpy as np
import time
from collections import deque

# Haar cascade paths
cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
eye_cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_eye.xml"

# Load classifiers
face_cascade = cv2.CascadeClassifier(cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

# Open webcam
cap = cv2.VideoCapture(0)
prev_time = time.time()
frame_count = 0

# Moving average buffer for smoothing pupil position
pupil_positions = deque(maxlen=5)

def detect_pupil(eye_roi):
    """Detect pupil using HoughCircles with improved preprocessing."""
    gray = cv2.cvtColor(eye_roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)  # Histogram equalization for better contrast
    gray = cv2.GaussianBlur(gray, (7, 7), 0)  # Reduce noise
    
    # Adaptive thresholding to handle lighting variations
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY_INV, 11, 2)
    
    # Detect circles using HoughCircles
    circles = cv2.HoughCircles(thresh, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=50, param2=20, minRadius=5, maxRadius=30)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            return (i[0], i[1], i[2])  # x, y, radius
    return None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame for faster processing
    frame = cv2.resize(frame, (640, 480))
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.2, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))

        for (ex, ey, ew, eh) in eyes:
            eye_roi = face_roi[ey:ey+eh, ex:ex+ew]
            pupil_center = detect_pupil(eye_roi)

            if pupil_center:
                px, py, pr = pupil_center
                pupil_positions.append((px, py))  # Store latest positions
                
                # Compute moving average of the last few positions
                avg_x = int(np.mean([pos[0] for pos in pupil_positions]))
                avg_y = int(np.mean([pos[1] for pos in pupil_positions]))
                
                cv2.circle(eye_roi, (avg_x, avg_y), pr, (0, 255, 0), 2)  # Draw smoothed pupil tracking

            cv2.rectangle(face_roi, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)  # Draw rectangle around eyes

    # Calculate frame rate
    frame_count += 1
    current_time = time.time()
    fps = frame_count / (current_time - prev_time)

    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Optimized Eye Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import cv2

cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
eye_cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_eye.xml"

face_cascade = cv2.CascadeClassifier(cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

#number signifies camera
cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    eyes = eye_cascade.detectMultiScale(gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(img,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        roi_gray2 = gray[ey:ey+eh, ex:ex+ew]
        roi_color2 = img[ey:ey+eh, ex:ex+ew]
        circles = cv2.HoughCircles(roi_gray2,cv2.HOUGH_GRADIENT,1,200,param1=200,param2=1,minRadius=0,maxRadius=0)
        try:
            for i in circles[0,:]:
                # draw the outer circle
                cv2.circle(roi_color2,(i[0],i[1]),i[2],(255,255,255),2)
                print("drawing circle")
                # draw the center of the circle
                cv2.circle(roi_color2,(i[0],i[1]),2,(255,255,255),3)
        except Exception as e:
            print(e)
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type

OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type

OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type

OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type

OpenCV(4.11.0) :-1: error: (-5:Bad argument)

2025-03-12 13:58:00.940 python[11351:286290] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-12 13:58:00.940 python[11351:286290] +[IMKInputSession subclass]: chose IMKInputSession_Modern


OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type

OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type

OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type

OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type

OpenCV(4.11.0) :-1: error: (-5:Bad argument)

KeyboardInterrupt: 

: 

In [5]:
import numpy as np
import cv2
import time

cap = cv2.VideoCapture(0) 	#640,480
w = 640
h = 480

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
	
		#downsample
		#frameD = cv2.pyrDown(cv2.pyrDown(frame))
		#frameDBW = cv2.cvtColor(frameD,cv2.COLOR_RGB2GRAY)
	
		#detect face
		frame = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
		faces = cv2.CascadeClassifier(eye_cascade_path)
		detected = faces.detectMultiScale(frame, 1.3, 5)
	
		#faces = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
		#detected2 = faces.detectMultiScale(frameDBW, 1.3, 5)
		
		pupilFrame = frame
		pupilO = frame
		windowClose = np.ones((5,5),np.uint8)
		windowOpen = np.ones((2,2),np.uint8)
		windowErode = np.ones((2,2),np.uint8)

		#draw square
		for (x,y,w,h) in detected:
			cv2.rectangle(frame, (x,y), ((x+w),(y+h)), (0,0,255),1)	
			cv2.line(frame, (x,y), ((x+w,y+h)), (0,0,255),1)
			cv2.line(frame, (x+w,y), ((x,y+h)), (0,0,255),1)
			pupilFrame = cv2.equalizeHist(frame[y+(h*.25):(y+h), x:(x+w)])
			pupilO = pupilFrame
			ret, pupilFrame = cv2.threshold(pupilFrame,55,255,cv2.THRESH_BINARY)		#50 ..nothin 70 is better
			pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_CLOSE, windowClose)
			pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_ERODE, windowErode)
			pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_OPEN, windowOpen)

			#so above we do image processing to get the pupil..
			#now we find the biggest blob and get the centriod
			
			threshold = cv2.inRange(pupilFrame,250,255)		#get the blobs
			contours, hierarchy = cv2.findContours(threshold,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
			
			#if there are 3 or more blobs, delete the biggest and delete the left most for the right eye
			#if there are 2 blob, take the second largest
			#if there are 1 or less blobs, do nothing
			
			if len(contours) >= 2:
				#find biggest blob
				maxArea = 0
				MAindex = 0			#to get the unwanted frame 
				distanceX = []		#delete the left most (for right eye)
				currentIndex = 0 
				for cnt in contours:
					area = cv2.contourArea(cnt)
					center = cv2.moments(cnt)
					cx,cy = int(center['m10']/center['m00']), int(center['m01']/center['m00'])
					distanceX.append(cx)	
					if area > maxArea:
						maxArea = area
						MAindex = currentIndex
					currentIndex = currentIndex + 1
		
				del contours[MAindex]		#remove the picture frame contour
				del distanceX[MAindex]
			
			eye = 'right'

			if len(contours) >= 2:		#delete the left most blob for right eye
				if eye == 'right':
					edgeOfEye = distanceX.index(min(distanceX))
				else:
					edgeOfEye = distanceX.index(max(distanceX))	
				del contours[edgeOfEye]
				del distanceX[edgeOfEye]

			if len(contours) >= 1:		#get largest blob
				maxArea = 0
				for cnt in contours:
					area = cv2.contourArea(cnt)
					if area > maxArea:
						maxArea = area
						largeBlob = cnt
					
			if len(largeBlob) > 0:	
				center = cv2.moments(largeBlob)
				cx,cy = int(center['m10']/center['m00']), int(center['m01']/center['m00'])
				cv2.circle(pupilO,(cx,cy),5,255,-1)

	
		#show picture
		cv2.imshow('frame',pupilO)
		cv2.imshow('frame2',pupilFrame)
		if cv2.waitKey(1) & 0xFF == ord('q'):
			break
	
	#else:
		#break

# Release everything if job is finished
cap.release()
cv2.destroyAllWindows()


TabError: inconsistent use of tabs and spaces in indentation (<string>, line 18)

In [1]:
import cv2
import numpy as np
import time

# Haar cascade paths
cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
eye_cascade_path = "/opt/anaconda3/envs/cv-env/share/opencv4/haarcascades/haarcascade_eye.xml"

# Load classifiers
face_cascade = cv2.CascadeClassifier(cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

# Open webcam
cap = cv2.VideoCapture(0)
frame_width = 640
frame_height = 480

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame for faster processing
    frame = cv2.resize(frame, (frame_width, frame_height))
    
    # Convert to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.2, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        gray_face = gray_frame[y:y+h, x:x+w]

        # Detect eyes inside the face region
        eyes = eye_cascade.detectMultiScale(gray_face, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))

        for (ex, ey, ew, eh) in eyes:
            eye_roi = face_roi[ey:ey+eh, ex:ex+ew]
            gray_eye = gray_face[ey:ey+eh, ex:ex+ew]

            # Apply histogram equalization for better contrast
            gray_eye = cv2.equalizeHist(gray_eye)

            # Apply adaptive thresholding
            pupil_frame = cv2.adaptiveThreshold(
                gray_eye, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
            )

            # Morphological operations to remove noise
            kernel = np.ones((3,3), np.uint8)
            pupil_frame = cv2.morphologyEx(pupil_frame, cv2.MORPH_CLOSE, kernel)
            pupil_frame = cv2.morphologyEx(pupil_frame, cv2.MORPH_OPEN, kernel)

            # Find contours
            contours, _ = cv2.findContours(pupil_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                # Find largest contour
                largest_contour = max(contours, key=cv2.contourArea)

                # Compute centroid of pupil
                M = cv2.moments(largest_contour)
                if M["m00"] != 0:  # Avoid division by zero
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    cv2.circle(eye_roi, (cx, cy), 5, (0, 255, 0), -1)  # Draw pupil center

            # Draw rectangle around the detected eye
            cv2.rectangle(face_roi, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)

    # Show frames
    cv2.imshow('Pupil Tracking', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam and close windows
cap.release()
cv2.destroyAllWindows()


2025-03-12 20:03:44.500 python[13906:341849] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-12 20:03:44.500 python[13906:341849] +[IMKInputSession subclass]: chose IMKInputSession_Modern
